In [8]:
# pip install pandas

In [9]:
# pip install pymongo

In [1]:
import pandas as pd
import json

import pymongo
import sys

from secrets1 import CLIENT_USERNAME, CLIENT_PASSWORD

dataset = pd.read_csv("car_prices.csv")
print("There are",dataset.shape[0],"instances")

There are 558837 instances


In [2]:
dataset['is_digit']= dataset['saledate'].str.isdigit().fillna(False)
dataset=dataset[dataset['is_digit']==False]
dataset=dataset.drop('is_digit', axis=1)

C:\Users\Emerson\AppData\Local\Temp\ipykernel_29228\1887482300.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['is_digit']= dataset['saledate'].str.isdigit().fillna(False)


In [3]:
dataset['saledate']=pd.to_datetime(dataset['saledate'])
dataset['salemonth_str']=dataset['saledate'].astype('str').str[:10]
dataset['salemonth']=pd.to_datetime(dataset['salemonth_str']).dt.strftime('%Y-%m')
dataset=dataset.drop('salemonth_str', axis=1)

C:\Users\Emerson\AppData\Local\Temp\ipykernel_29228\282653654.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataset['saledate']=pd.to_datetime(dataset['saledate'])
C:\Users\Emerson\AppData\Local\Temp\ipykernel_29228\282653654.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  dataset['saledate']=pd.to_datetime(dataset['saledate'])


In [5]:
dataset.dtypes

year              int64
make             object
model            object
trim             object
body             object
transmission     object
vin              object
state            object
condition       float64
odometer        float64
color            object
interior         object
seller           object
mmr             float64
sellingprice    float64
saledate         object
salemonth        object
dtype: object

In [6]:
dataset['state'] = dataset['state'].apply(lambda x: x.upper())

In [8]:
json_data = dataset.to_json(orient='records')
with open('json_data.json', 'w') as json_file:
    json_file.write(json_data)

item_dict = json.loads(json_data)
print(len(item_dict))

558811


In [11]:
try:
    client = pymongo.MongoClient("mongodb+srv://"+ CLIENT_USERNAME 
                                + ":" + CLIENT_PASSWORD + "@bdtt-finalproject.75mthdj.mongodb.net/")

# return a friendly error if a URI error is thrown 
except pymongo.errors.ConfigurationError:
    print("An Invalid URI host error was received. Is your Atlas host name correct in your connection string?")
    sys.exit(1)

db = client.myDatabase
my_collection = db['car_sale_info']

# drop the collection in case it already exists
try:
    my_collection.drop()  

# return a friendly error if an authentication error is thrown
except pymongo.errors.OperationFailure:
    print("An authentication error was received. Are your username and password correct in your connection string?")
    sys.exit(1)

try: 
    result = my_collection.insert_many(item_dict)

# return a friendly error if the operation fails
except pymongo.errors.OperationFailure:
    print("An authentication error was received. Are you sure your database user is authorized to perform write operations?")
    sys.exit(1)
else:
    inserted_count = len(result.inserted_ids)
    print("I inserted %d documents." % inserted_count)

    print("\n")

An authentication error was received. Are you sure your database user is authorized to perform write operations?


AttributeError: 'tuple' object has no attribute 'tb_frame'